In [0]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 281.3/281.3 MB 72.1 MB/s eta 0:00

*** WARNING: max output size exceeded, skipping output. ***

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 918.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 27.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=deefe2d4b736fffe678ef0bb2888e4a02214d2aeb940ffd0625ea32b5e267d62
  Stored in directory: /root/.cache/pip/wheels/1d/27/68/1382001655ef41217e1dd34d59aa777612135379bab64279e9
Successfully built pyspark


In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

In [0]:
from datetime import datetime,date
import pandas as pd
from pyspark.sql import Row


In [0]:
#Using sparl
df=spark.createDataFrame([
    Row(a=1,b=2),
    Row(a=12,b=22),
    Row(a=13,b=21)
])

In [0]:
df.show()

+---+---+
|  a|  b|
+---+---+
|  1|  2|
| 12| 22|
| 13| 21|
+---+---+



In [0]:
#Using Pandas
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
pandas_df

,a,b,c,d,e
0,1,2.0,string1,2000-01-01,2000-01-01 12:00:00
1,2,3.0,string2,2000-02-01,2000-01-02 12:00:00
2,3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [0]:
df=spark.createDataFrame(pandas_df)
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [0]:
#rdd to dataframe
rdd = spark.sparkContext.parallelize([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
])
df = spark.createDataFrame(rdd, schema=['a', 'b', 'c', 'd', 'e'])
df

Out[8]: DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [0]:
type(df.c)

Out[9]: pyspark.sql.column.Column

In [0]:
df.select(df.c).show()

+-------+
|      c|
+-------+
|string1|
|string2|
|string3|
+-------+



In [0]:
df.select(df.c).show()

+-------+
|      c|
+-------+
|string1|
|string2|
|string3|
+-------+



In [0]:
df.filter(df.a == 1).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+



In [0]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    return series +1

df.select(pandas_plus_one(df.a)).show()

+------------------+
|pandas_plus_one(a)|
+------------------+
|                 2|
|                 3|
|                 4|
+------------------+



In [0]:
@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    # Simply divide col a by one by using pandas Series.
    return series /2

df.select(pandas_plus_one(df.a)).show()

+------------------+
|pandas_plus_one(a)|
+------------------+
|                 0|
|                 1|
|                 1|
+------------------+



In [0]:
#adding 2   columns
@pandas_udf('long')
def pandas_plus_one1(series1: pd.Series, series2: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    return series1 + series2

df.select(pandas_plus_one1(df.a, df.b)).show()

+----------------------+
|pandas_plus_one1(a, b)|
+----------------------+
|                     3|
|                     5|
|                     7|
+----------------------+



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()



In [0]:
data=[("Z", 1),("A", 20),("B", 30),("C", 40),("B", 30),("B", 60)]
inputRDD = spark.sparkContext.parallelize(data)
listRdd = spark.sparkContext.parallelize([1,2,3,4,5,3,2])

In [0]:
seqOp2 = (lambda x, y: (x[0] + y, x[1] + 1))
combOp2 = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
agg2=listRdd.aggregate((0, 0), seqOp2, combOp2)
print(agg2) 


(20, 7)
